In [1]:
import random
import time

nobn = 512
nobk = 10
sl = 2

def eucledianGCD(a,b):
    if a == 0:
        return b
    return eucledianGCD(b%a,a)

def squareAndMultiply(a,b,n):
    y = bin(int(b))
    x = 1
    for i in range(2,len(y)):
        x = (x*x)%n
        if int(y[i]) == 1:
            x = (x*a)%n
    return x

def millerRabinPrimalityTest(n,k):
    d = n-1
    r = 0
    while d%2==0:
        d = d//2
        r = r+1
    d = int(d) 
    flag = 0
    for i in range(k):
        a = random.randint(2,n-2)
        x = squareAndMultiply(a,d,n)
        if x == 1 or x == n-1:
            continue
        
        for j in range(r-1):
            flag = 0
            x = ((x%n)*(x%n))%n
            if x == n-1:
                flag = 1
                break
        if flag == 1:
            continue
        return False
    return True

class Peggy:
    def __init__(self,k):
        self.k = k
        self.p = 0
        self.q = 0
        self.S = []
        self.sgn = 0
        self.r = 0
        
    def generatePrimes(self):
        print("\nGenerating two large primes p and q ...")
        while True:
            self.p = random.getrandbits(nobn)
            i = random.getrandbits(nobk)
            if self.p>3 and i>0 and millerRabinPrimalityTest(self.p,i):
                break
    
        while True:
            self.q = random.getrandbits(nobn)
            i = random.getrandbits(nobk)
            if self.q>3 and i>0 and millerRabinPrimalityTest(self.q,i) and self.q != self.p:
                break
                
        print("Two primes are p = "+str(self.p)+" q = "+str(self.q))
        N = self.p*self.q
        print("N = "+str(N))
        return N
    
    def generatePrivateKey(self,N):
        print("\nGenerating private key of Peggy ...")
        for i in range(self.k):
            x = 0
            while True:
                x = random.randint(1,N-1)
                if x<N and eucledianGCD(x,N) == 1 and x not in self.S:
                    break
            self.S.append(x)
        print("Private Key of Peggy = "+str(self.S))
    
    def generatePublicKey(self,N):
        print("\nGenerating public key of Peggy ...")
        V = []
        for i in range(self.k):
            v = (self.S[i]*self.S[i])%N
            V.append(v)
        print("Public Key of Peggy = "+str(V))
        return V
    
    def generateX(self,N):
        print("\nGenerating X = s*r^2 mod N")
        self.r = random.randint(1,N-1)
        self.sgn = self.sgn = random.choice((-1,1))
        X = (self.sgn * (((self.r%N)*(self.r%N))%N))%N
        print("r = "+str(self.r))
        print("sgn = "+str(self.sgn))
        print("X = "+str(X))
        return X
        
    def computeY(self,Ai,N):
        print("\nComputing Y = r*((s1)^a1)*((s2)^a2)*...*((sk)^ak) mod N")
        Y = self.r
        for i in range(self.k):
            if Ai[i] != 0:
                Y = (Y*self.S[i])%N
        print("Y = "+str(Y))
        return Y
    
class Victor:
    def __init__(self,k):
        self.k = k 
        
    def generateChallenge(self):
        print("\nGenerating challenge key ...")
        Ai = []
        for i in range(self.k):
            a = random.getrandbits(1)
            Ai.append(a)
        print("A = "+str(Ai))
        return Ai
    
    def check(self, Y, X, A, N, V):
        print("\nChecking y^2 = +-x*((v1)^a1)*((v2)^a2)*...*((vk)^ak) mod N and x =/= 0")
        Y_square_n = (Y*Y)%N
        temp1 = 1
        temp2 = -1
        for i in range(self.k):
            if A[i] != 0:
                temp1 = (temp1*V[i])%N
                temp2 = (temp2*V[i])%N
        temp1 = (temp1*X)%N
        temp2 = (temp2*X)%N
        
        print("y^2 mod N = "+str(Y_square_n))
        print("x*((v1)^a1)*((v2)^a2)*...*((vk)^ak) mod N = "+str(temp1))
        print("-x*((v1)^a1)*((v2)^a2)*...*((vk)^ak) mod N = "+str(temp2))
        
        if (Y_square_n == temp1 or Y_square_n == temp2) and X != 0:
            return True
        return False
    
def main():
    k = int(input("Enter the value of k (Length of Secret): "))
    t = int(input("Enter the value of t (Number of Iterations): "))
    peggy = Peggy(k)
    N = peggy.generatePrimes()
    time.sleep(sl)
    peggy.generatePrivateKey(N)
    time.sleep(sl)
    V = peggy.generatePublicKey(N)
    time.sleep(sl)
    victor = Victor(k)
    flag = 1
    
    for i in range(t):
        print("\nIteration Number = "+str(i+1))
        X = peggy.generateX(N)
        time.sleep(sl)
        A = victor.generateChallenge()
        time.sleep(sl)
        Y = peggy.computeY(A,N)
        time.sleep(sl)
        res = victor.check(Y,X,A,N,V)
        time.sleep(sl)
        if res == False:
            flag = 0
            break
    if flag == 1:
        print("\nPeggy Passed")
    else:
        print("\nPeggy Lied")

main()

Enter the value of k (Length of Secret): 3
Enter the value of t (Number of Iterations): 5

Generating two large primes p and q ...
Two primes are p = 9958967947107031074235876927525756387526374954395001239179376814030769270836813748646232581304967607880877941607985934929417657270562304847614321718644603 q = 3929634905295622397054211339442289220534118160170800622841929297895490057161178533567383486448121738398410020068270643985539748003109382566118639643377491
N = 39135108065672077056946786911283959969161679968955695893445900532390896358061684276732038487485372327348783633948267652542000502506010027560419904367470602644524682334325924683196600804555252427440869914138942456892824135450461745625520399242714970808881648054026298402523867941012653267809218063689598831073

Generating private key of Peggy ...
Private Key of Peggy = [390752512186945839319735277188810665060174436433002627030860169812046934690351702234484517188406575328319420761342646342507028150004270913433903316620450844452940


Checking y^2 = +-x*((v1)^a1)*((v2)^a2)*...*((vk)^ak) mod N and x =/= 0
y^2 mod N = 623788478456088770587441305038467599723756914384530745792978821761603577923064027676012064470965166078290197058000675638183914713992322770846566955028158074267229401815290152813581000338454749893442721747093779971617148891689935824866719107602563518286786106106792423344526544495319316308911229991648145508
x*((v1)^a1)*((v2)^a2)*...*((vk)^ak) mod N = 38511319587215988286359345606245492369437923054571165147652921710629292780138620249056026423014407161270493436890266976903816587792017704789573337412442444570257452932510634530383019804216797677547427192391848676921206986558771809800653680135112407290594861947919505979179341396517333951500306833697950685565
-x*((v1)^a1)*((v2)^a2)*...*((vk)^ak) mod N = 6237884784560887705874413050384675997237569143845307457929788217616035779230640276760120644709651660782901970580006756381839147139923227708465669550281580742672294018152901528135810003384547498934427217470937799